In [1]:
# 必要なライブラリをインストール
!pip install dalex          # DALEX本体：XAIの主役
!pip install xgboost        # XGBoost：高性能な機械学習モデル
!pip install pandas         # pandas：データを扱うための基本ツール
!pip install numpy          # numpy：数値計算の基礎ライブラリ
!pip install scikit-learn   # scikit-learn：機械学習の定番ライブラリ
!pip install matplotlib     # matplotlib：グラフ描画用

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for dalex: filename=dalex-1.7.2-py3-none-any.whl size=1042898 sha256=59391e4df01f3d41e4493093c2957c3853933ad113f77dd55f4c9de50acca764
  Stored in directory: /root/.cache/pip/wheels/17/0a/3a/8c988bdca6acd7ce4aa949726b26c382ad1635e065bf83a175
Successfully built dalex


In [2]:
# ライブラリをインポート（道具箱から必要な道具を取り出すイメージ）
import dalex as dx
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')  # 不要な警告を非表示に
# バージョン確認（正しくインストールされたかチェック）
print(f"DALEX version: {dx.__version__}")

DALEX version: 1.7.2


In [4]:
# タイタニックデータセットを読み込む
titanic = dx.datasets.load_titanic()
# データの中身を確認
print("最初の5行を表示：")
print(titanic.head())
print(f"\nデータの形状: {titanic.shape}")
print(f"行数（乗客数）: {titanic.shape[0]}, 列数（特徴量）: {titanic.shape[1]}")
# 各列の意味を理解しよう
print("\n各列の説明：")
print("- survived: 生存したか（1=生存、0=死亡）")
print("- class: チケットクラス（1st, 2nd, 3rd）")
print("- gender: 性別")
print("- age: 年齢")
print("- sibsp: 同乗した兄弟・配偶者の数")
print("- parch: 同乗した親・子供の数")
print("- fare: 運賃")
print("- embarked: 乗船港")

最初の5行を表示：
   gender   age class     embarked   fare  sibsp  parch  survived
0    male  42.0   3rd  Southampton   7.11      0      0         0
1    male  13.0   3rd  Southampton  20.05      0      2         0
2    male  16.0   3rd  Southampton  20.05      1      1         0
3  female  39.0   3rd  Southampton  20.05      1      1         1
4  female  16.0   3rd  Southampton   7.13      0      0         1

データの形状: (2207, 8)
行数（乗客数）: 2207, 列数（特徴量）: 8

各列の説明：
- survived: 生存したか（1=生存、0=死亡）
- class: チケットクラス（1st, 2nd, 3rd）
- gender: 性別
- age: 年齢
- sibsp: 同乗した兄弟・配偶者の数
- parch: 同乗した親・子供の数
- fare: 運賃
- embarked: 乗船港


In [5]:
# データの前処理
# カテゴリ変数（文字データ）を数値に変換
X = pd.get_dummies(titanic.drop('survived', axis=1))  # 説明変数（予測に使う情報）
y = titanic['survived']  # 目的変数（予測したい結果）

In [6]:
# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(
    X, # 説明変数
    y, # 目的変数
    test_size=0.25, # テストデータの割合
    random_state=42
)
print(f"訓練データ: {X_train.shape[0]}件")
print(f"テストデータ: {X_test.shape[0]}件")

訓練データ: 1655件
テストデータ: 552件


In [7]:
# XGBoostモデルを構築
model = xgb.XGBClassifier(
    n_estimators=100,     # 決定木の数（多いほど複雑だが時間もかかる）
    max_depth=3,          # 各決定木の深さ（深いほど複雑）
    learning_rate=0.1,    # 学習の速度（小さいほど慎重に学習）
    random_state=42       # 再現性のための種
)
# モデルを訓練（学習させる）
model.fit(X_train, y_train)
# モデルの性能を評価
y_pred = model.predict(X_test)
accuracy = (y_pred == y_test).mean()
print(f"予測精度（Accuracy）: {accuracy:.4f}")

予測精度（Accuracy）: 0.7899


In [9]:
# Explainerオブジェクトを作成
explainer = dx.Explainer(
    model,                        # 解釈したいモデル
    X_test,                       # テストデータ（説明変数）
    y_test,                       # テストデータ（目的変数）
    label='XGBoost生存予測モデル'    # モデルの名前（グラフに表示される）
)
# Explainerの基本情報を確認
print("Explainerの準備完了！")
print(f"モデルタイプ: {explainer.model_type}")
print(f"データサイズ: {explainer.data.shape}")

Preparation of a new explainer is initiated

  -> data              : 552 rows 17 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 552 values
  -> model_class       : xgboost.sklearn.XGBClassifier (default)
  -> label             : XGBoost生存予測モデル
  -> predict function  : <function yhat_proba_default at 0x7e424328c7c0> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.023, mean = 0.329, max = 0.994
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.99, mean = -0.0106, max = 0.919
  -> model_info        : package xgboost

A new explainer has been created!
Explainerの準備完了！
モデルタイプ: classification
データサイズ: (552, 17)


In [10]:
# 変数重要度を計算
vi = explainer.model_parts()
# 結果を表示
print("変数重要度の数値：")
print(vi.result)

変数重要度の数値：
                  variable  dropout_loss           label
0      embarked_Queenstown      0.191094  XGBoost生存予測モデル
1                    parch      0.193794  XGBoost生存予測モデル
2   class_engineering crew      0.195851  XGBoost生存予測モデル
3       embarked_Cherbourg      0.196019  XGBoost生存予測モデル
4              gender_male      0.196937  XGBoost生存予測モデル
5             _full_model_      0.196937  XGBoost生存予測モデル
6   class_victualling crew      0.196952  XGBoost生存予測モデル
7     embarked_Southampton      0.197910  XGBoost生存予測モデル
8         embarked_Belfast      0.198626  XGBoost生存予測モデル
9                class_2nd      0.200678  XGBoost生存予測モデル
10                   sibsp      0.206558  XGBoost生存予測モデル
11  class_restaurant staff      0.207689  XGBoost生存予測モデル
12               class_1st      0.207701  XGBoost生存予測モデル
13                    fare      0.210209  XGBoost生存予測モデル
14         class_deck crew      0.228761  XGBoost生存予測モデル
15               class_3rd      0.229477  XGBoost生存予測モデル
16                   

In [11]:
vi.plot(show=True)

In [12]:
# テストデータから一人の乗客を選択
passenger_index = 0  # 最初の乗客を選択
single_passenger = X_test.iloc[[passenger_index]]
# この乗客の情報を確認
print("選択した乗客の情報：")
for col, val in single_passenger.iloc[0].items():
    print(f"  {col}: {val}")

選択した乗客の情報：
  age: 20.0
  fare: 0.0
  sibsp: 0
  parch: 0
  gender_female: True
  gender_male: False
  class_1st: False
  class_2nd: False
  class_3rd: False
  class_deck crew: False
  class_engineering crew: False
  class_restaurant staff: True
  class_victualling crew: False
  embarked_Belfast: False
  embarked_Cherbourg: False
  embarked_Queenstown: False
  embarked_Southampton: True


In [13]:
# 予測結果
prediction = model.predict_proba(single_passenger)[0][1]
actual = y_test.iloc[passenger_index]
print(f"モデルの予測生存確率: {prediction:.2%}")
print(f"実際の結果: {'生存' if actual == 1 else '死亡'}")

モデルの予測生存確率: 65.71%
実際の結果: 生存


In [14]:
# 予測の要因分解
bd = explainer.predict_parts(
    single_passenger,
    type='break_down'  # 段階的に要因を分解する手法
)
# 結果を可視化
bd.plot(show=True)